# Расчет плотности населения

## Общая информация
Для расчета плотности населения используем данные из [kontur](https://data.humdata.org/dataset/kontur-population-dataset).

Это датасет, который состоит из векторных шестиугольники H3 с подсчетом численности населения с разрешением 400 м. 
Кроме того, для более быстрого вычисления плотности можно использовать датасеты с меньшим разрешением 3 км и 22 км соответственно:
[kontur-population-dataset-3km](https://data.humdata.org/dataset/kontur-population-dataset-3km) и [kontur-population-dataset-22km](https://data.humdata.org/dataset/kontur-population-dataset-22km). 
А также можно брать плотность населения для отдельной страны: [https://data.humdata.org/organization/kontur](https://data.humdata.org/organization/kontur).

## Сам код

#### Подгрузка population данных

In [4]:
import h3
import geopandas as gpd

In [5]:
data_path_input = "../data/"
kontur_url_400_RU = data_path_input + "kontur_population_RU_20231101.gpkg"

In [6]:
kontur_df_400 = gpd.read_file(kontur_url_400_RU)

In [7]:
kontur_df_400.head()

,h3,population,geometry
0,88316ebaebfffff,46.0,"POLYGON ((14625017.829 5294766.727, 14624986.9..."
1,88316ebae3fffff,81.0,"POLYGON ((14624549.047 5293694.850, 14624518.0..."
2,88316ebad5fffff,10.0,"POLYGON ((14627640.639 5293931.207, 14627609.8..."
3,88316ebacdfffff,207.0,"POLYGON ((14625955.795 5296910.957, 14625924.9..."
4,88316ebac9fffff,20.0,"POLYGON ((14626986.716 5296989.945, 14626955.9..."


In [8]:
kontur_df_400.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

#### Подгрузим DataFrame с административным делением Москвы

In [9]:
mo_data_url = data_path_input+'mo.geojson'

In [10]:
mp_gdf = gpd.read_file(mo_data_url)

In [11]:
mp_gdf.head()

,NAME,OKATO,OKTMO,NAME_AO,OKATO_AO,ABBREV_AO,TYPE_MO,geometry
0,Киевский,45298555,45945000,Троицкий,45298000,Троицкий,Поселение,"MULTIPOLYGON (((36.80310 55.44083, 36.80319 55..."
1,Филёвский Парк,45268595,45328000,Западный,45268000,ЗАО,Муниципальный округ,"POLYGON ((37.42765 55.74821, 37.42849 55.74875..."
2,Новофёдоровское,45298567,45954000,Троицкий,45298000,Троицкий,Поселение,"POLYGON ((36.80357 55.45162, 36.80451 55.46551..."
3,Роговское,45298575,45956000,Троицкий,45298000,Троицкий,Поселение,"POLYGON ((36.93724 55.24139, 36.93726 55.24161..."
4,"""Мосрентген""",45297568,45953000,Новомосковский,45297000,Новомосковский,Поселение,"POLYGON ((37.43956 55.62731, 37.44018 55.63042..."


In [12]:
mp_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Видим, что данные границ районов и плотности населения в разных системах координат. 
Поэтому переведем все в одну систему координат. Датафрейм с районами Москвы находится 
в EPSG:4326 системе координат, которая является самой простой, но недостаточно точной 
для России. Датафрейм с плотностью населения находится WSG-84. Наиболее точной и подходящей 
будет ESPG:32637.

In [13]:
kontur_df_400.to_crs('epsg:32637', inplace=True)
mp_gdf.to_crs('epsg:32637', inplace=True)

Теперь вычислим плотность каждого района

In [14]:
mp_gdf['density'] = mp_gdf.geometry.apply(lambda x: kontur_df_400[kontur_df_400.within(x)].population.mean())

In [15]:
mp_gdf.head()

,NAME,OKATO,OKTMO,NAME_AO,OKATO_AO,ABBREV_AO,TYPE_MO,geometry,density
0,Киевский,45298555,45945000,Троицкий,45298000,Троицкий,Поселение,"MULTIPOLYGON (((361024.612 6146042.994, 361033...",186.833333
1,Филёвский Парк,45268595,45328000,Западный,45268000,ЗАО,Муниципальный округ,"POLYGON ((401303.896 6179176.113, 401357.980 6...",7602.750000
2,Новофёдоровское,45298567,45954000,Троицкий,45298000,Троицкий,Поселение,"POLYGON ((361092.260 6147242.530, 361200.499 6...",105.858407
3,Роговское,45298575,45956000,Троицкий,45298000,Троицкий,Поселение,"POLYGON ((368851.532 6123593.484, 368853.528 6...",66.228814
4,"""Мосрентген""",45297568,45953000,Новомосковский,45297000,Новомосковский,Поселение,"POLYGON ((401748.704 6165705.878, 401795.516 6...",6326.000000


In [16]:
with open(data_path_input+'population_density.geojson', 'w') as file:
    file.write(mp_gdf.to_json())